In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('flite_price_cleaned_data.csv')
data.head()

,Unnamed: 0,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Price
0,0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,non-stop,3897
1,1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,2 stops,7662
2,2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,2 stops,13882
3,3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,1 stop,6218
4,4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,1 stop,13302


In [3]:
data=data.drop(labels=['Unnamed: 0'],axis=1)

In [4]:
x=data.drop(labels=['Price'],axis=1)
y=data['Price']
y

0         3897
1         7662
2        13882
3         6218
4        13302
         ...  
10678     4107
10679     4145
10680     7229
10681    12648
10682    11753
Name: Price, Length: 10683, dtype: int64

In [5]:
x

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Total_Stops
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,non-stop
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,2 stops
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,2 stops
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,1 stop
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,1 stop
...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,19:55,22:25,non-stop
10679,Air India,27/04/2019,Kolkata,Banglore,20:45,23:20,non-stop
10680,Jet Airways,27/04/2019,Banglore,Delhi,08:20,11:20,non-stop
10681,Vistara,01/03/2019,Banglore,New Delhi,11:30,14:10,non-stop


In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()

In [7]:
encoder.fit_transform(x[['Airline','Date_of_Journey','Source','Destination','Dep_Time','Arrival_Time','Total_Stops']]).toarray()

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [8]:
x=pd.DataFrame(encoder.fit_transform(x[['Airline','Date_of_Journey','Source','Destination','Dep_Time','Arrival_Time','Total_Stops']]).toarray(),
               columns=encoder.get_feature_names_out())

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.20)
x_train.shape,x_test.shape

((8546, 1637), (2137, 1637))

In [10]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [11]:
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [12]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [13]:
def evaluate_model(true,predict):
    mse=mean_squared_error(true,predict)
    score=r2_score(true,predict)
    return mse,score

In [16]:
models={
#     'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train_scaled,y_train)
    y_pred=model.predict(x_test_scaled)
    mse,r2_square=evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Training Performance')
    print("MSE:",mse)
    
    print("R2 score",r2_square)
    r2_list.append(r2_square)
    print('='*30)

C:\Users\aman_\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.373e+08, tolerance: 1.819e+07
  model = cd_fast.enet_coordinate_descent(


Lasso
Model Training Performance
MSE: 4391221.612720921
R2 score 0.79251423853317
Ridge
Model Training Performance
MSE: 4437724.12758629
R2 score 0.790316988984431
Elasticnet
Model Training Performance
MSE: 5004076.044132274
R2 score 0.7635567912475809


In [17]:
import pickle
pickle.dump(scaler,open('flite_scaler.pkl','wb'))
pickle.dump(Ridge,open('flite_ridge.pkl','wb'))